<h1 style='color: #00A78C; font-size: 34px; font-weight: bold;'>Analise Vendas Delivery


</h1>


# <font color="#FFA17A" style="font-size: 30px;">Objetivo</font>
<hr style="border: 2px solid #FFA17A;">

 <p style='font-size: 23px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i><b>Utilizar comandos SQL para realizar análises conjuntas em um banco de dados com informações diversas sobre vendas de sucos</b></i>     
</p>

---

In [3]:
# importando bibliotecas necessárias

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
from sqlalchemy import create_engine, MetaData, Table, select

In [28]:
!pip install pymysql

In [43]:
caminhos = [os.path.join("Tabelas", "channels.csv"),
os.path.join("Tabelas", "deliveries.csv"),
os.path.join("Tabelas", "drivers.csv"),
os.path.join("Tabelas", "hubs.csv"),
os.path.join("Tabelas", "payments.csv"),
os.path.join("Tabelas", "orders.csv"),
os.path.join("Tabelas", "stores.csv")]

dataframes = []


In [101]:
canais =  pd.read_csv(caminhos[0])
entregas = pd.read_csv(caminhos[1])
motoristas = pd.read_csv(caminhos[2])
centros_de_distribuicao = pd.read_csv(caminhos[3], encoding='latin-1')
pagamentos = pd.read_csv(caminhos[4])
pedidos = pd.read_csv(caminhos[5])
lojas =pd.read_csv(caminhos[6],  encoding='latin-1')

In [106]:
display(canais.columns)
display(entregas.columns)
display(motoristas.columns)
display(centros_de_distribuicao.columns)
display(pagamentos.columns)
display(pedidos.columns)
display(lojas.columns)

Index(['channel_id', 'channel_name', 'channel_type'], dtype='object')

Index(['delivery_id', 'delivery_order_id', 'driver_id',
       'delivery_distance_meters', 'delivery_status'],
      dtype='object')

Index(['driver_id', 'driver_modal', 'driver_type'], dtype='object')

Index(['hub_id', 'hub_name', 'hub_city', 'hub_state', 'hub_latitude',
       'hub_longitude'],
      dtype='object')

Index(['payment_id', 'payment_order_id', 'payment_amount', 'payment_fee',
       'payment_method', 'payment_status'],
      dtype='object')

Index(['order_id', 'store_id', 'channel_id', 'payment_order_id',
       'delivery_order_id', 'order_status', 'order_amount',
       'order_delivery_fee', 'order_delivery_cost', 'order_created_hour',
       'order_created_minute', 'order_created_day', 'order_created_month',
       'order_created_year', 'order_moment_created', 'order_moment_accepted',
       'order_moment_ready', 'order_moment_collected',
       'order_moment_in_expedition', 'order_moment_delivering',
       'order_moment_delivered', 'order_moment_finished',
       'order_metric_collected_time', 'order_metric_paused_time',
       'order_metric_production_time', 'order_metric_walking_time',
       'order_metric_expediton_speed_time', 'order_metric_transit_time',
       'order_metric_cycle_time'],
      dtype='object')

Index(['store_id', 'hub_id', 'store_name', 'store_segment', 'store_plan_price',
       'store_latitude', 'store_longitude'],
      dtype='object')

In [107]:
engine = create_engine('sqlite:///:memory:')

In [108]:
canais.to_sql('canais',engine,index=False)
entregas.to_sql('entregas',engine,index=False)
motoristas.to_sql('motoristas',engine,index=False)
centros_de_distribuicao.to_sql('centros_de_distribuicao',engine,index=False)
pagamentos.to_sql('pagamentos',engine,index=False)
pedidos.to_sql('pedidos',engine,index=False)
lojas.to_sql('lojas',engine,index=False)

951

In [109]:
inspector = inspect(engine)
print(inspector.get_table_names())

['canais', 'centros_de_distribuicao', 'entregas', 'lojas', 'motoristas', 'pagamentos', 'pedidos']


# <font color="#FFA17A" style="font-size: 30px;">Análise de Dados</font>
<hr style="border: 2px solid #FFA17A;">

In [111]:
# Função que irá fazer a consulta SQL e retornar o dataframe

def sql_df(query):
  with engine.connect() as conexao:
    consulta = conexao.execute(text(query))
    dados = consulta.fetchall()
  return pd.DataFrame(dados,columns=consulta.keys())

In [122]:
query = '''SELECT * FROM ENTREGAS'''

sql_df(query).head(4)

,delivery_id,delivery_order_id,driver_id,delivery_distance_meters,delivery_status
0,2174658,68413340,8378.0,5199.0,DELIVERED
1,2174660,68414309,2473.0,410.0,DELIVERED
2,2174661,68416230,7615.0,3784.0,DELIVERED
3,2174663,68412721,8378.0,5714.0,DELIVERED


## Quais motoristas pecorreram maiores distâncias

In [142]:
query = '''SELECT  MOTORISTAS.driver_id, MOTORISTAS.driver_modal, SUM(ENTREGAS.delivery_distance_meters) AS DISTANCIA_TOTAL 
FROM MOTORISTAS
JOIN ENTREGAS ON ENTREGAS.driver_id = MOTORISTAS.driver_id
GROUP BY MOTORISTAS.driver_id
ORDER BY DISTANCIA_TOTAL DESC
'''

dist_motoristas = sql_df(query)
dist_motoristas.head(4)

,driver_id,driver_modal,DISTANCIA_TOTAL
0,26223,MOTOBOY,60504610.0
1,25651,MOTOBOY,32466086.0
2,15898,MOTOBOY,8209339.0
3,17749,MOTOBOY,8060048.0


---
## Os entregadores que percorreram as maiores distâncias são os de ID igual a '26223', '25651', '15898'. Todos eles utilizam o motocicletas como veículos
---